<a href="https://colab.research.google.com/github/stmulugheta/20223-ML-DL-models/blob/master/PyTorch_Lab_GNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DM Lab on GNN with PyTorch and PyTorch Geometric

Lab notebook by Andrea Mastropietro, inspire by content [here](https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8).

In this lab we will see how to handle graph data and feed them to GNNs. We will use PyG, a library that stands upon PyTorch for working with graph data and graph neural networks. It allows yoiu to oute out-of-the-shelf graph layers or to define your own by using the basic components of graph neural networks, message passing, propagation.

Install neeeded libraries woth the correct versions of CUDA and PyTorch

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

## Data Example: What does PyG want?

![picture](https://drive.google.com/uc?id=1BD2-3qqzb_WsY2swP8VGNqJVSsWgqYjV)

(image from [here](https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8))

We want to create the graph we see above using PyTorch tensors

In [ ]:
import torch

We have four nodes with feature vectors and labels

In [ ]:
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

We need now to define the edges.

PyG wants edges in COO format (coordinate format): two lists, the first one with soruce nodes and the second one with target nodes. The order of the edge index does not matter, we need only to know which are the edges, there is no ordering among them.

In [ ]:
edge_index = torch.tensor([[0, 1, 2, 0, 3],
                           [1, 0, 1, 3, 2]], dtype=torch.long)

Let's define a Data object for our PyTorch model

In [ ]:
from torch_geometric.data import Data

data = Data(x=x, y=y, edge_index=edge_index)

We can convert the ataset to networkX graph in order to visualzie it

In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx

G = to_networkx(data, to_undirected=False)
nx.draw(G, with_labels=True)

## Karate Club - Node Classification with Predefined GCNConv layers

We will know see how to perfomr basic node classification by creating a neural netowk using the layers altredy implemented by PyG. We will biuld a 2-layered GCN. The reference paper for this model is [here](https://arxiv.org/abs/1609.02907).

We will use a simple netwrok, Zachary's Karate. The nodes represent a member of the club and two memeber are connected if they interacted. The club was split due to internat conlfilts, and the labels indicate if e member followed the instruction, "Mr Hi", or not. This is a social interaction dataset. This is a very small dataset, we do not expect amazing results :)

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# load graph from networkx library
G = nx.karate_club_graph()
print(nx.info(G))

nx.draw(G, with_labels=True)

In [ ]:
# retrieve the labels for each node
labels = np.asarray([G.nodes[i]['club'] != 'Mr. Hi' for i in G.nodes]).astype(np.int64)

# create edge index. We need to have data as previously shown. We can exploit networkX and scipy for that 
adj = nx.to_scipy_sparse_matrix(G).tocoo() #coordinate format
print(adj)

In [ ]:
#create edge index in the proper way
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)

display(edge_index)

In [ ]:
# using degree as embedding. For simplicity, the feature vector describing the 
# will be just its degree, which is enough for us
embeddings = np.array(list(dict(G.degree()).values()))

# normalizing degree values
scale = StandardScaler()
embeddings = scale.fit_transform(embeddings.reshape(-1,1))
print(embeddings)

### Create Custom Dataset

In order to create datastets with PyG you can use InMemoryDataset and Dataset. Similar to TorchVision, the first one is used for data that fit in main memory, the second one can hendel larger datasets by reading batches from your disk. We will use the fisrt one now, but the implementation is similar.

We will extend the class InMemoryDataset to create our own dataset class.

In [ ]:
import pandas as pd
from torch_geometric.data import InMemoryDataset
from sklearn.model_selection import train_test_split
import torch_geometric.transforms as T

# custom dataset
class KarateDataset(InMemoryDataset):
    def __init__(self, transform=None):
        super(KarateDataset, self).__init__('.', transform, None, None) #pre transform and pre filter: None, we don't need them

        data = Data(edge_index=edge_index)
        
        data.num_nodes = G.number_of_nodes()
        
        # embedding 
        data.x = torch.from_numpy(embeddings).type(torch.float32)
        
        # labels
        y = torch.from_numpy(labels).type(torch.long)
        data.y = y.clone().detach() #removing tensors computational graph for efficency since it is not needed
        
        data.num_classes = 2

        # splitting the data into train, validation and test
        X_train, X_test, y_train, y_test = train_test_split(pd.Series(G.nodes()), 
                                                            pd.Series(labels),
                                                            test_size=0.30, 
                                                            random_state=42)
        
        n_nodes = G.number_of_nodes()
        
        # create train and test masks for data
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[X_train.index] = True
        test_mask[X_test.index] = True
        data['train_mask'] = train_mask
        data['test_mask'] = test_mask

        self.data, self.slices = self.collate([data])

    # def _download(self):
    #     return

    # def _process(self):
    #     return

    # def __repr__(self):
    #     return '{}()'.format(self.__class__.__name__)
    
dataset = KarateDataset()
data = dataset[0]

In [ ]:
data

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# GCN model with 2 layers 
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_features, 16) #in feature, out dim
        self.conv2 = GCNConv(16, int(data.num_classes))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data =  data.to(device)

model = Net().to(device) 

In [ ]:
import sys

torch.manual_seed(42)

#optimizer_name = "Adam"
lr = 1e-1
#optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 200

def train():
  model.train()
  optimizer.zero_grad()
  #negative log likelihood, we need it after a softmax output activation function
  #we also tell to compute the loss using the  mask (so, on the train samples) 
  F.nll_loss(model(data)[data.train_mask], data.y[data.train_mask]).backward() 
  optimizer.step()
  
  # comment the following for avoiding eval during training
  model.eval()
  logits = model(data)
  train_mask = data['train_mask']
  train_pred = logits[train_mask].max(1)[1]
  train_acc = train_pred.eq(data.y[train_mask]).sum().item() / train_mask.sum().item() #eq: elementwise equality

  return train_acc
  

@torch.no_grad()
def test():
  model.eval()
  logits = model(data)

  # uncomment the following if you want to eval on the train and test together
  # train_mask = data['train_mask']
  # train_pred = logits[train_mask].max(1)[1]
  # train_acc = train_pred.eq(data.y[train_mask]).sum().item() / train_mask.sum().item()

  test_mask = data['test_mask']
  test_pred = logits[test_mask].max(1)[1]
  test_acc = test_pred.eq(data.y[test_mask]).sum().item() / test_mask.sum().item()

  return test_acc

from tqdm import tqdm, tqdm_notebook

for epoch in tqdm_notebook(range(1, epochs)):
  train_acc = train()

  

test_acc = test()

print('#' * 70)
print('Train Accuracy: %s' % train_acc)
print('Test Accuracy: %s' % test_acc)
print('#' * 70)

## Explainability - GNNExplainer

why was a node classified as this or that? Explainability methods try to tell us that. GNNExplainer is the first exmplainability method for GNNs. It retrieves an explanatipn graph by finding the most importnat edges for the given prediction.

We will usethe PyG implemantation rather that the official one. The corresponding paper is [here](https://arxiv.org/abs/1903.03894).

In [ ]:
import os.path as osp

import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.nn import GCNConv, GNNExplainer

torch.manual_seed(42)

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, normalize=False)
        self.conv2 = GCNConv(16, dataset.num_classes, normalize=False)

    def forward(self, x, edge_index, edge_weight):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = data.to(device)
lr = 1e-1

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 200

x, edge_index, edge_weight = data.x, data.edge_index, None

num_epochs = 200

for epoch in tqdm(range(num_epochs)):
    model.train()
    optimizer.zero_grad()
    log_logits = model(x, edge_index, edge_weight)
    loss = F.nll_loss(log_logits[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

with torch.no_grad():
  model.eval()
  logits = model(x, edge_index, edge_weight)

  #test_mask = data['test_mask']
  preds = logits.max(1)[1]


In [ ]:
explainer = GNNExplainer(model, epochs=200, return_type='log_prob', num_hops = 2) #if num_hops is none it is detected from the num of message passing ops.
                                                                                  #it is needed to tell the explainer "how far to go" to look for explanations.
node_idx = 10 #node to explain 0, 10... node 0 is Mr Hi
pred_node_idx = preds[node_idx].item()
print("Explaining node ", node_idx, " with predicted class: ", pred_node_idx)

node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edge_index,
                                                   edge_weight=edge_weight)

In [ ]:
ax, G = explainer.visualize_subgraph(node_idx, edge_index, edge_mask, y=data.y, seed = node_idx, threshold = None) #you can set threshold to define the hard mask accordign to the sparisty you wnat ot obtain and how much you want to be strict
plt.show()

NOTE: you have directions now even if the original graph was undirected because, when creating the edge mask, you can consider the infomration flow from both directions of the message passing operation. We have to say that subgraphs retrieved by GNNExplainer are not guaranteed to be connected! If you consider this propery desirable for your application, then your choice could be SubgraphX ;) Since a GNN aggregate info up to N hops, even edges not connected to the explainee node are meaningful, since the info actually arrives to our node! So we see which are the most imporatn connections for the prediction, with aggregate info arrives to our node.

Usually, when exlaining node 0 (Mr. Hi) no real explanation info in conveyed, since he is the founder of the new club and he moslty influenced others.

# Custon Model Definition - Node Classification with GraphSAGE on RecSYS datasets

We will see how to build a GraphSAGE layer, from the correspoding [paper](https://arxiv.org/abs/1706.02216). This model is already implemented in PyG, but we will build it from stratch to see that we can define our own GNN conv layers :)

We will use data from the 2015 Recommander Systems Challenge 

## Data Loading

We will load the data and define a custom Data class, as before.

Those data are divived into two csvs yoochoose-clicks.dat, and yoochoose-buys.dat, containing click events and buy events, respectively and we need  to see if a click event was followedby a buy event. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from sklearn.preprocessing import LabelEncoder

#your GDrive path here :)
CLICKS_PATH = "/content/gdrive/MyDrive/University/PhD/Teaching/2021-2022/Data Mining/Labs/Lab 4 - GNNs with PyG/data/yoochoose-clicks.dat"
BUYS_PATH = "/content/gdrive/MyDrive/University/PhD/Teaching/2021-2022/Data Mining/Labs/Lab 4 - GNNs with PyG/data/yoochoose-buys.dat"

df_clicks = pd.read_csv(CLICKS_PATH, header=None) 
display(df_clicks.head())
df_clicks.columns=['session_id','timestamp','item_id','category']

df_buys = pd.read_csv(BUYS_PATH, header=None) 
df_buys.columns=['session_id','timestamp','item_id','price','quantity']

item_encoder = LabelEncoder()
df_clicks['item_id'] = item_encoder.fit_transform(df_clicks.item_id)
display(df_clicks.head())

In [ ]:
#we taka a random sample of the dataset since it is to large
import numpy as np

sampled_session_id = np.random.choice(df_clicks.session_id.unique(), 10000, replace=False)
df_clicks = df_clicks.loc[df_clicks.session_id.isin(sampled_session_id)]
print(df_clicks.nunique())

To determine the ground truth we simply check if a session_id in yoochoose-clicks presents in yoochoose-buys (so if a click event led to a buy event)

In [ ]:
df_clicks['label'] = df_clicks.session_id.isin(df_buys.session_id)
df_clicks.head()

### Building the Dataset

We tranform our data to a  Dataset object. We treat each item in a session as a node: all items in the same session form a graph. Thus, we need to group the data by session_id and iterate over these groups. In each iteration, the item_id in each group are categorically encoded again since for each graph, the node index should count from 0.

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class YooChooseDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDataset, self).__init__(root, transform, pre_transform)

        self.data, self.slices = torch.load(self.processed_paths[0])

    #It returns a list that shows a list of raw, unprocessed file names
    @property
    def raw_file_names(self):
        return []

    #returns a list containing the file names of all the processed data    
    @property
    def processed_file_names(self):
        return ['processed_data.dat']

    #This function should download the data you are working on to the directory as specified in self.raw_dir we don't need it
    def download(self):
        pass

    #This is the most important method of Dataset. You need to gather your data into a list of Data objects. Then, call self.collate() to compute the slices that will be used by the DataLoader object.
    # We need this since we are working with data consituted by multiple sessions/graphs, so we need to collate them into a single Data objects  
    def process(self):
        
        data_list = []

        # process by session_id
        grouped = df_clicks.groupby('session_id')
        for session_id, group in tqdm(grouped):
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes,
                                   target_nodes], dtype=torch.long)
            x = node_features

            y = torch.FloatTensor([group.label.values[0].astype(int)])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

We shuffle the data and split into train, test and validation sets. We do it outside the data class now to see a different approach, but you can do it the same way as before by defining masks.

In [ ]:
dataset = YooChooseDataset(root = "../")

In [ ]:
dataset

In [ ]:
#data = dataset[0]
dataset = dataset.shuffle()
train_data = dataset[:8000]
val_data = dataset[8000:9000]
test_data = dataset[9000:]
len(train_data), len(val_data), len(test_data)

We define a DataLoader

In [ ]:
from torch_geometric.loader import DataLoader
batch_size= 512
train_loader = DataLoader(train_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
num_items = df_clicks.item_id.max() + 1 
num_items

### Custom Layer Definition - GraphSAGE

Here comes the hard part :) Let's build our custom  layer as a GraphSAGE layer.

In [ ]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "max" aggregation. This is the aggregation type used in GraphSAGE
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

### Network Definition using GraphSAGE layer

We now build the whole GrapHSAGE network from the reference paper using the layer just defined.

In [ ]:
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F

embed_dim = 128

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=num_items, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _ , _= self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

Define optimizer and loss criterion

In [ ]:
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()

Define train and test functions

We use the roc auc as metric, since the dataset is higly umbalanced and using accuracy could be not really meaninful.

In [ ]:
from sklearn.metrics import roc_auc_score

def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_data)


def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return roc_auc_score(labels, predictions)

In [ ]:
for epoch in tqdm(range(10)):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

## Extra - GNNExplainer on Cora Dataset


In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GNNExplainer

dataset = 'Cora'
path = osp.join(osp.dirname(osp.realpath("content/")), '..', 'data', 'Planetoid')
transform = T.Compose([T.GCNNorm(), T.NormalizeFeatures()])
dataset = Planetoid(path, dataset, transform=transform)
data = dataset[0]


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, normalize=False)
        self.conv2 = GCNConv(16, dataset.num_classes, normalize=False)

    def forward(self, x, edge_index, edge_weight):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight

num_epochs = 200

for epoch in tqdm(range(num_epochs)):
    model.train()
    optimizer.zero_grad()
    log_logits = model(x, edge_index, edge_weight)
    loss = F.nll_loss(log_logits[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

explainer = GNNExplainer(model, epochs=200, return_type='log_prob')
node_idx = 10 #node to explain
node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edge_index,
                                                   edge_weight=edge_weight)
ax, G = explainer.visualize_subgraph(node_idx, edge_index, edge_mask, y=data.y)
plt.show()